In [1]:
import numpy as np
import pandas as pd
import csv

from sqlalchemy import create_engine

# Extract

### Extract CSVs into DataFrames

In [5]:
imdb_path = "resources/imdb_top_1000.csv"
academy_award_path = "resources/academy_awards.csv"

imdb_df = pd.read_csv(imdb_path)
academy_award_df = pd.read_csv(academy_award_path)

# Transform

### Transform IMDB dataframe

### Transform Academy Award dataframe

# Load

### Create database connection

In [ ]:
connection_string = "postgres:postgres@localhost:5432/movies_project_db"
engine = create_engine(f'postgresql://{connection_string}')

### Load DataFrames into database

In [ ]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [ ]:
county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)